In [188]:
import requests as r
import pandas as pd
from bs4 import BeautifulSoup

%matplotlib inline

In [174]:
url = 'https://results.thecaucuses.org/'

In [175]:
resp = r.get(url)

In [176]:
soup = BeautifulSoup(resp.text)

In [177]:
precinct_table = soup.find('div', {'class': 'precinct-table'})
thead = [li.text for li in soup.find('ul', {'class': 'thead'}).findAll('li')]
sub_head = [li.text for li in soup.find('ul', {'class': 'sub-head'}).findAll('li')]

In [178]:
candidates = [thead[i] for i in range(2,len(thead), 3)]

In [179]:
counties = precinct_table.findAll('div', attrs = {'class': 'precinct-rows'})

In [180]:
county = counties[0]

In [181]:
rows = []
for county in counties:
    county_name = county.find('div', {'class': 'precinct-county'}).find('div', {'class': 'wrap'}).text
    county_data = county.find('div', {'class': 'precinct-data'}).findAll('ul', lambda v: v != 'total-row')
    for row in county_data:
        row_data = [col.text for col in row.findAll('li')]
        precinct_name = row_data[0]
        for i in range(1,len(row_data), 3):
            first_exp, final_exp, sde = row_data[i:(i+3)]
            candidate = candidates[i//3]
            rows.append(dict(county=county_name, precinct=precinct_name, candidate=candidate, 
                             first_exp=int(first_exp), final_exp=int(final_exp), sde=float(sde)))

In [182]:
df = pd.DataFrame(rows).sort_values(['county', 'precinct', 'candidate'])

In [194]:
df.groupby(['candidate', 'county'])[['first_exp', 'final_exp', 'sde']].sum()

first_exp  final_exp     sde
candidate county                                  
Bennet    Adair               0          0  0.0000
          Adams               0          0  0.0000
          Allamakee           1          0  0.0000
          Appanoose           0          0  0.0000
          Audubon             0          0  0.0000
...                         ...        ...     ...
Yang      Winnebago          30         29  0.4500
          Winneshiek        151         49  0.5120
          Woodbury          127         40  0.6375
          Worth              27         23  0.4000
          Wright              5          0  0.0000

[1386 rows x 3 columns]

In [195]:
df.to_csv('iowa_data.csv', index=False)